In [12]:
import csv
import pandas as pd
import glob
import os

from xlsxwriter.workbook import Workbook
import sys
import xlsxwriter
from tkinter import filedialog
from tkinter import *
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipywidgets import GridspecLayout
from IPython.display import HTML
import datetime as dt
from datetime import datetime
import re
dir_=""
def p(i):
    liste=[0,1,2,3,4,12,5,6,7,8,9,10,13]
    return liste[i]
def combine_csv(dir_):
    
    mydir=dir_
    files=glob.glob(os.path.join(mydir, "*.csv"))
    dataframes = []

    for csvfile in files:
        df=pd.read_csv(csvfile)
        print(csvfile)
        dataframes.append(df)

    result=pd.concat(dataframes, ignore_index=True)
    result.to_csv(mydir+'\\all.csv', index=False, encoding="utf-8-sig")

    f=open(mydir+"\\all.csv", encoding="utf8")
    FORMAT = '%d.%m.%Y'
    workbook=xlsxwriter.Workbook(mydir+"\\"+"Patientbook_"+datetime.now().strftime(FORMAT)+".xlsx")
    worksheet=workbook.add_worksheet()
    rem_words=["med", "Diplom Designer","Herr ","Dipl Designer","dent","rerpol", "MBA", "Dr. med.", "Dr.", "Dipl. Ing. ", "Prof. Dr. ", "Prof", ".", "DiplIng", "Dipl- Ing.", "Dipl-Ing"]
    i=1
    try:
        reader=csv.reader(f)
        worksheet.write(0,0, "Name")
        worksheet.write(0,1, "Vorname")
        worksheet.write(0,2,"Umsatz")
        worksheet.write(0,3,"Ziffern")
        worksheet.write(0,4,"Katalog")
        worksheet.write(0,5,"Alter")
        worksheet.write(0,6,"Arzt")
        worksheet.write(0,7,"Leistungstag1")
        worksheet.write(0,8,"PLZ")
        worksheet.write(0,9,"Ort")
        worksheet.write(0,10,"Leistungstag2")
        worksheet.write(0,11,"Name Vorname")
        worksheet.write(0,12,"Versicherung")
        worksheet.write(0,13,"Praxis")
        re_test_patienten=re.compile(".*_Test")
        for row in reader:
            liste=[]
            if i > 1:
                nachname=row[0].split(", ")[0]
                if re_test_patienten.match(nachname):
                    continue
                vorname=row[0].split(", ")[1]
                for item in rem_words: 
                    nachname=nachname.replace(item,"")
                nachname=nachname.strip()
                liste.append(nachname)
                liste.append(vorname)
                for n in range(10):
                    liste.append(row[n+1].strip())
                liste.append(row[11].split(",")[0])
                j=0
                for eintrag in liste:
                    if j == 6:
                        worksheet.write_number(i,p(j),int(eintrag))
                    elif j == 2:
                        worksheet.write_number(i,p(j),float(eintrag.replace(',', '.')))
                    else:
                        worksheet.write(i,p(j),eintrag)
                    j=j+1
            i=i+1
    finally:
        f.close()
        workbook.close()
    print("finished writing xlsx file!")

def remove_num(nr, line):
        return re.sub(".*"+nr, '', line).strip()
def write_line(worksheet,key_list, i):
    for j in range(14):
        worksheet.write(i, j, key_list.get(j))

def conv_date(tag):
    re_day = re.compile("\d{6}")
    if re_day.match(tag):
        date = dt.datetime.strptime(tag, '%Y%m%d').date()
        return date
    else:
        return ""
def comb_con(dataframes, worksheet, workbook):
    dict_num = {
            "3101":"Name des Patienten",
            "3102":"Vorname des Patienten",
            "3103":"Geburtsdatum des Patienten",
            "3110":"Geschlecht des Patienten",
            "3112":"PLZ des Patienten",
            "3113" :"Wohnort",
            "4134":"Versicherung",
            "5000":"Leistungstag",
            "5001":"Gebührennummer",
            "5098":"BSNR",
            "0185099":"Arzt",
    }

    new_dict = {
            "3101":"Name des Patienten",
            "3102":"Vorname des Patienten",
            "3110":"Geschlecht des Patienten",
            "3112":"PLZ des Patienten",
            "5098":"BSNR",
            "0185099":"Arzt",

    }
    dict_index = {
            "3101":0,
            "3102":1,
            "3103":2,
            "3110":3,
            "3112":4,
            "3113":5,
            "4134":6,
            "5000":7,
            "5001":8,
            "5098":9,
            "0185099":10,
            "4121" : 11

    }

   

    keyliste = list(dict_num.keys())
    new_keyliste = list(new_dict.keys())
    j = 0
    for key in keyliste:
        inhalt=dict_num.get(key)
        worksheet.write(0, j, inhalt)
        j = j + 1
    
    data=""
    for file in dataframes:
        with open(file) as f:
            data+=f.read().strip()
        data+="\n"
    with open("combined_file.txt", "w") as final:
        final.write(data)
        final.close()
    with open("combined_file.txt") as c_file:
        key_list = dict.fromkeys(list(range(10)))
        i = 1
        j = 1
        vorname = ""
        nachname = ""
        tag = ""
        leistung = ""
        wohnort = ""
        date = "None"
        geb = "None"
        versicherung = ""
        gebuhren_nr = ""
        doc = ""
        for line in c_file:
            line=line.strip()
            re_geb = re.compile("\d\d\d3103")
            re_nr = re.compile("\d\d\d3000")
            re_wohnort = re.compile("\d\d\d3113")
            re_tag = re.compile("\d\d\d5000")
            re_leistung = re.compile("\d\d\d5001")
            re_gebuhr = re.compile("\d\d\d4121")
            re_doc = re.compile("0185099.*")
            re_ver = re.compile("\d\d\d4134")
            re_name = re.compile("\d\d3101")
            re_vorname = re.compile("\d\d3102")

    #             writing the formulas
#             worksheet.write_formula(f'N{j}', f'=SUBSTITUTE(A{j}, " ", "")&SUBSTITUTE(B{j}, " ", "")')
#             worksheet.write_formula(f'L{j}', f'=VLOOKUP(O{j}, Patientenbuch!$L$1:$M$5000, 2, FALSE)')
#             worksheet.write_formula(f'M{j}', f'=VLOOKUP(I{j}, EBM_Katalog!$C$6:$D$3200,2,FALSE)')
            if re_geb.match(line):
                geb = str(conv_date(remove_num("3103", line)))
                worksheet.write(j, dict_index.get("3103"), geb)

            if re_doc.match(line):
                doc = remove_num("0185099", line)
                worksheet.write(j-1, dict_index.get("0185099"), doc)

            if re_tag.match(line):
                tag = remove_num("5000", line)
                date = str(conv_date(tag))
                worksheet.write(j, dict_index.get("5000"), date)


            if re_ver.match(line):
                versicherung = remove_num("4134", line)
                if versicherung == "":
                    worksheet.write(j, dict_index.get("4134"), "keine Versicherung")
                else:
                    worksheet.write(j, dict_index.get("4134"), versicherung)

            if re_leistung.match(line):
                if remove_num("5001",line).isnumeric():
                        worksheet.write_number(j, dict_index.get("5001"), int(remove_num("5001", line)))
                else:
                    worksheet.write(j, dict_index.get("5001"), remove_num("5001", line))
                worksheet.write(j, dict_index.get("5000"), date)

                worksheet.write(j, dict_index.get("3113"), wohnort)
                worksheet.write(j, dict_index.get("3103"), geb)
                worksheet.write(j, dict_index.get("4121"), gebuhren_nr)
                worksheet.write(j, dict_index.get("0185099"), doc)
                if versicherung == "":
                    worksheet.write(j, dict_index.get("4134"), "keine Versicherung")
                else:
                    worksheet.write(j, dict_index.get("4134"), versicherung)
                write_line(worksheet,key_list, j)
                j = j + 1

            if re_wohnort.match(line):
                wohnort = remove_num("3113", line)
                worksheet.write(i, dict_index.get("3113"), wohnort)

            if re_nr.match(line):
                key_list.clear()
                key_list = dict.fromkeys(list(range(12)))
                i = j

                vorname = ""
                nachname = ""
                tag = ""
                leistung = ""
                wohnort = ""
                date = "None"
                geb = "None"
                versicherung = ""
                gebuhren_nr = ""
                doc = ""

            else:
                for key in new_keyliste:
                    reg = re.compile("\d\d\d"+key)
                    if reg.match(line):
                        key_list[dict_index.get(key)] = remove_num(key, line)
                        inhalt=remove_num(key,line)
                        if inhalt.isnumeric():
                            worksheet.write_number(i,dict_index.get(key), int(inhalt))
                        else:
                            worksheet.write(i, dict_index.get(key), inhalt)




def start_con(worksheet, workbook):
    files=glob.glob(os.path.join(dir_, "*.CON"))
    dataframes=[]
    for confile in files:
        dataframes.append(confile)

    comb_con(dataframes, worksheet, workbook)
def start_csv(worksheet):
    pass



def change_dir(b):
    root = Tk()
    root.withdraw()
    folder_selected = filedialog.askdirectory()
    print("Changed directory to: " + folder_selected)
    global dir_
    dir_=folder_selected
def start_both(b):
    con_filename="Export_CON_"+datetime.now().strftime(FORMAT)+".xlsx"
    wb_excel = xlsxwriter.Workbook(dir_+"/"+con_filename)
    ws_con = wb_excel.add_worksheet()
    print("started writing CON...")
    start_con(ws_con, wb_excel)
    wb_excel.close()
    print("...finished writing CON")
    print("started writing csv-File...")
    combine_csv(dir_)
    print("...finished writing csv-File")


    
    
p1=progress=widgets.FloatProgress(value=0.0, min=0.0, max=1.0)
p2=progress=widgets.FloatProgress(value=0.0, min=0.0, max=1.0)
c1=widgets.Valid(description='File uploaded',value=False,)
c2=widgets.Valid(description='File downloaded',value=False,)

b1=widgets.Button(description='2. Start',disabled=False)
b2=widgets.Button(description='1. Change Directory', disabled=False)
FORMAT = '%d.%m.%Y'
formatted_date1="Export_CON_"+datetime.now().strftime(FORMAT)+".xlsx"
formatted_date2="Patientbook_"+datetime.now().strftime(FORMAT)


b1.on_click(start_both)
b2.on_click(change_dir)
grid=GridspecLayout(2,2)



grid[1,0]=b1
grid[0,0]=b2




display(grid)    





GridspecLayout(children=(Button(description='2. Start', layout=Layout(grid_area='widget001'), style=ButtonStyl…

Changed directory to: C:/Users/kai.follmann/Downloads/CONTEST
started writing CON...
...finished writing CON
started writing csv-File...
C:/Users/kai.follmann/Downloads/CONTEST\Patientenbuch_20210903_1527 (2).csv


IndexError: list index out of range